In [1]:
import numpy as np
import astropy.units as u
import re
import os
import skvideo.io
import math
import sunpy.coordinates
import shutil
from datetime import date, time, datetime, timedelta
from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
from PIL import Image, ImageDraw

# Generating Disks (title)

### Author: Alex Feghhi 
### Created: 
### Last run: 

### Goals:
- Reads in ...
- Output

Where/how you got the fits files (specific config, query)

Separate notebook for how to extract metadata from fits files into one txt, then reference the notebook here

In [ ]:
data = open("C:/Users/alexf/Desktop/hmi_Mharp_720s_2010.04.01_TAI-2010.12.31_TAI.txt","r")
workdir = 'C:/Users/alexf/Desktop/HMI_Data/'


Directory to store output files...

In [ ]:
Ydata_dir =  workdir + 'Ydata/'
if not os.path.exists(Ydata_dir):
    os.mkdir(Ydata_dir)
    print("Directory " + Ydata_dir + "does not exist. Creating...")

In [2]:
lines = data.readlines()
del lines[0] #if inputfile has no characters first line(like T_REC), remove this line
alldata = np.zeros((5,len(lines))) #change 100000 to len(lines) to do full year
# FIXME transpose alldata?
for x in range(len(lines)): #change 100000 to len(lines) to do full year
    # row_info = lines[x].split() # FIXME
    # col_dict['HARPnum'] rather than 0
    alldata[0,x] = lines[x].split()[0] #HARPnum
    alldata[1,x] =  (re.sub("[^0-9]", "", (lines[x].split()[1]))) #T_REC
    alldata[2,x] = lines[x].split()[4] #LAT
    alldata[3,x] = lines[x].split()[5] #LON
    # size assumed to be in pixels for this code, NACR is in pixels but SIZE_ACR is in mH
    alldata[4,x] = lines[x].split()[15] #SIZE_ACR MAYBE UNPROJECTED VS PROJECTED    
    

In [3]:
size_dict = {alldata[0, 0]:alldata[4,0]}
for i in range(1,alldata.shape[1]):#loop over alldata to generate a max size dictionary
    if(alldata[0,i] != alldata[0,i-1]):
        size_dict[alldata[0,i]] = alldata[4,i]#size_acr
    else:
        if(alldata[4,i] > size_dict[alldata[0, i]]):
            size_dict[alldata[0,i]] = alldata[4, i]
alldata = alldata[:,alldata[1,:].argsort()]#sort the array by time

Explain choice of dates (small sample to test code?)

Chose time period in demo video (include link)

In [4]:
start = datetime(2010, 5, 1,0,0,0)#date time object format is year, month, day, hour, minute, second
end = datetime(2010,5,31,23,0,0)#the end time is included amongst disks generated
time_interval = timedelta(minutes = 60)

# FIXME original size 4096, say downsample to 3 different pixel scales

output_sizes = [256, 512, 1024]
for size in output_sizes:
    resize_dir = Ydata_dir + str(size)
    if  os.path.exists(resize_dir):#delete any resizing directories matching the new resizes
        shutil.rmtree(resize_dir)
    os.makedirs(resize_dir)


In [ ]:
# FIXME explain rounding, include metadata keys from fits files
# we need pixel scale up to 5 decimal digits
# x, y centers up to integer precision
pixel_scale = 0.504 # arcsec/pix
x_center = 2037 # pix
y_center = 2050 # pix

number_disks = int((end-start)/time_interval + 1)
for i in range(number_disks):
    time = start + i*time_interval
    timestring = int(time.strftime('%Y' + '%m' + '%d' + '%H' + '%M' + '%S'))
    obstimestring = time.strftime('%Y' + '-' + '%m' + '-' + '%d')
    indices = np.where(alldata[1] == timestring)[0]
    disk = Image.new('1', (4096, 4096), color='white') # black/white canvas, initialized to zeros
    draw = ImageDraw.Draw(disk)
    for index in indices:
        hpc1 = SkyCoord(alldata[3,index]*u.deg, alldata[2,index]*u.deg, frame=frames.HeliographicStonyhurst, obstime=obstimestring)
        # Stonyhurst --> arcsec
        hpc_out = sunpy.coordinates.Helioprojective(observer='earth', obstime=obstimestring) # arcsec
        hpc2 = hpc1.transform_to(hpc_out) # convert to arcsecond
        xc = (hpc2.Tx / u.arcsec) # stripping units
        yc = (hpc2.Ty / u.arcsec)
        # FIXME use variables
        xc = (xc/.504) + 2037 # convert to pixel value
        yc = (yc/.504) + 2050
        size = size_dict[alldata[0, index]]#replace this with alldata[4, index] if you don't want to use max size
        # FIXME say area to radius conversion, arbitrary scaling by 10
        radius = math.sqrt(size / math.pi) * 10
        draw.ellipse((xc-radius,yc-radius,xc+radius,yc+radius), 'black')
    for size in output_sizes:
        data = np.flipud(np.array(disk.resize((size,size), Image.BILINEAR))).astype(int)#remove.astype(int) to save as boolean
        np.save(Ydata_dir + str(size) + '/' + time.strftime('%Y' + '%m' + '%d' + '_' + '%H' +'%M' + '%S') + '_' + str(size), data)


In [5]:
#create video function with (start,end,size)
def generate_video(start,end,size):
    video_dir = Ydata_dir + str(size) + '/'
    array_filenames = os.listdir(video_dir)
    start_index = array_filenames.index(start.strftime('%Y' + '%m' + '%d' + '_' + '%H' +'%M' + '%S') + '_' + str(size) + '.npy')
    end_index = array_filenames.index(end.strftime('%Y' + '%m' + '%d' + '_' + '%H' +'%M' + '%S') + '_' + str(size) + '.npy')
    outputdata = np.zeros((end_index + 1 - start_index, size, size))
    index = 0
    for filename in array_filenames[start_index:end_index + 1]:
        outputdata[index,:,:] = np.load(video_dir + filename) * 255
        index = index + 1
    skvideo.io.vwrite(video_dir + 'outputvideo_'+str(size) +'.mp4', outputdata)

for size in output_sizes:
    generate_video(start,end,size)

In [6]:
number_disks = int((end-start)/time_interval + 1)
for i in range(number_disks):
    time = start + i*time_interval
    timestring = int(time.strftime('%Y' + '%m' + '%d' + '%H' + '%M' + '%S'))
    obstimestring = time.strftime('%Y' + '-' + '%m' + '-' + '%d')
    indices = np.where(alldata[1] == timestring)[0]
    disk = Image.new('1',(4096,4096),color = 'white')
    draw = ImageDraw.Draw(disk)
    for index in indices:
        hpc1 = SkyCoord(alldata[3,index]*u.deg, alldata[2,index]*u.deg, frame=frames.HeliographicStonyhurst, obstime=obstimestring)
        hpc_out = sunpy.coordinates.Helioprojective(observer = "earth", obstime=obstimestring)
        hpc2 = hpc1.transform_to(hpc_out)#convert to arcsecond
        xc = (hpc2.Tx / u.arcsec)
        yc = (hpc2.Ty / u.arcsec)
        xc = (xc/.504) + 2037#convert to pixel value
        yc = (yc/.504) + 2050
        size = size_dict[alldata[0,index]]#replace this with alldata[]
        print(xc,yc)
        radius = math.sqrt(size / math.pi) * 10
        draw.ellipse((xc-radius,yc-radius,xc+radius,yc+radius),'black')

344.4608972206729 2836.977167821563
2262.5077421822116 1257.7330501742886
1980.6070894891568 1171.4242237418957
1172.5035755934264 2670.1037408184675
1197.8366586262946 2655.067506918862
346.89331565263 2835.74039981827
1999.5605547212867 1174.0719965847138
2282.095593967464 1257.3303718998363
2299.9028181282583 1258.7585896006658
1214.9811368847472 2651.365405459454
2015.4934260442728 1174.6380097495053
349.8910967622005 2835.626452404111
1233.736429790024 2651.014709830941
352.6224609972794 2838.581640896685
2311.146332572109 1260.1486437158705
2030.8263924121884 1174.5433066339638
1251.144358605592 2650.243092492705
2049.1125574636667 1175.898478297736
357.709935034399 2841.0395345327024
2323.057221839802 1261.85032510279
2340.2698349875573 1262.4606907893358
2065.4429933081897 1175.3036416577374
361.5409327870286 2843.3868899803233
1265.3198865712684 2649.520807087824
2347.523025939406 1264.431639427537
1284.6945600356787 2646.9936963073055
2081.003630865866 1174.5279216406673
366.

2158.115700335787 3393.3482306610804
2116.557979400898 2592.334254881136
1606.4266729482738 1318.9981834044574
3016.497787087984 1233.5680477590222
2747.669032542465 1136.8576547603939
2172.0213701621906 3393.6523269367763
739.7538259939417 2884.4057865749382
3262.9199200910107 2717.767058226904
2136.0137972942052 2592.5353653741104
1614.206531835064 1321.4962597472027
1626.3126470536215 1322.6615709309085
2184.266028158433 3393.6694682621765
749.7853241474895 2885.565582394071
3276.1140524303237 2716.0827783517825
255.5857297939233 1434.8244020087254
2162.740180261376 2580.755186445861
2762.091571515046 1135.7995150562983
255.9685011641293 1435.5389546973583
2198.1717200598437 3393.2262940655637
1637.307491584772 1323.871990182328
2184.5663538531053 2572.7566609214073
759.7152212434637 2886.3149984761853
2775.112667672502 1135.4966727189426
3288.842749813553 2714.252571103647
2197.1790084462177 2570.810618616211
2788.536260022328 1134.4406004868042
769.4511595407525 2887.4135171489183

3601.8539315948956 2677.34359551667
2106.72889196546 1325.5931585019275
2605.1956656810216 3375.989791618798
2749.312626660945 2582.654632061516
2120.6428220605326 1325.6924878547393
1150.4113777523648 2912.5870660164
3613.0519557221915 2675.911155389519
3169.7445578575794 1107.948316098883
348.7409324433668 1476.0174368043668
2133.9972551679343 1326.3150894712508
3622.423591772137 2674.5246986845605
2764.8765026799147 2580.7315044147194
3180.5807865539828 1106.3296470002203
1167.6454700999698 2914.6676338619486
2615.585804254244 3375.343195266267
353.93157333428894 1478.970144693691
1180.901751067785 2914.0796813772104
3628.7359329300043 2674.603296818483
2149.325168244175 1326.440018667096
360.76348226554387 1478.5760805839536
2625.2503567880844 3375.2521882495275
3191.635474112156 1105.6840025141144
2788.5476893255604 2588.2979372379914
3638.792493483991 2673.752835574961
3201.587688297061 1105.7684459298603
2806.089535215184 2586.9265140937214
1196.0320957174397 2915.05080855021
26

2733.013092444702 2505.1381873215423
2331.022316834923 2833.628825432974
2950.8891985172822 3356.446794028726
2693.864225559826 1304.7823730311902
2753.6407594939774 2502.084248348224
1657.4598658587365 2938.8871473038416
3803.2304244653583 2646.433215682802
643.814469436342 1485.7954615022788
3806.9287701879935 2645.113248195891
2773.5538433464017 2501.643947097081
2710.563663701371 1303.25524067697
2346.4735700526285 2834.77548412825
655.6335599098486 1486.6165744583354
2963.246184886134 3355.6103751181736
1673.203989153468 2938.737862853124
667.208667728019 1488.1275512259804
2792.2966623776333 2501.0444256757064
2731.2981196967175 1302.3144632958997
2974.807862593053 3354.9867503718688
1691.7749538251537 2939.8542078599507
2362.0437663755633 2835.340533847124
3807.99968784296 2645.579449855599
677.9095769857913 1488.545480193967
2746.8716235079764 1302.3801815096988
2810.752418825741 2499.232510425729
2378.1686280889653 2836.2023824070816
3810.455922059573 2644.6963179329114
2982.4

3286.8748007984204 3322.2079029997567
2353.781007660936 2946.6581367610243
2861.3597715143824 1468.902692354619
1233.543329060611 1505.1909759485898
2877.6387300584256 1470.1977133876467
3294.6282945585954 3321.3300233293357
2370.4895981892846 2946.1377385578844
3296.4282646496567 3322.2575669645857
3293.9252509027265 1258.5425328096385
1249.2709456067143 1506.3474918954132
2389.233863936778 2945.5393539323695
2893.5431041728984 1469.887823490216
2402.917317026186 2947.3927367212727
3301.883543574009 3322.3054715086128
2908.5409272957313 1469.2080364176043
1263.5399590012619 1507.5290249656755
2921.946068058403 1467.5845607399597
3305.9545336826013 3321.204150603975
2419.9365055341937 2946.943924864483
1281.6259488640894 1507.8151763281248
3313.0793644608807 3318.022277994294
2936.862543329288 1463.997283916791
2434.645382642276 2942.7075147153446
1298.0066424280799 1505.8635182250925
1314.2912557488294 1506.951000340574
2951.872929687607 1463.9079384658198
3317.141288958861 3317.99887

...

1237.581746658096 2717.3943998738255
2567.181578979472 1610.5352025083073
2679.167865752597 2618.285782699558
1253.9374151034488 2717.872994359563
2584.17915987107 1610.3131618056796
2691.691777135894 2619.382822443311
2830.159736385547 2743.508174025439
3731.2080931497458 1583.059099246104
2601.597266541625 1610.1838193969388
2849.2665415120036 2743.209088076752
1272.6394949037426 2717.711707142145
2709.5807663156156 2618.697904022561
2729.918149684746 2618.2953070184185
2618.4645502805424 1610.2151114169776
1288.2272452781222 2717.190476572237
3734.373344400949 1581.4439319883004
2864.188017887058 2742.1494202365247
3737.6839385540525 1580.4210157296745
2747.861384538634 2617.5039604454782
2879.9357604762627 2739.5686539253343
2635.4652555299726 1610.0445715830451
1301.8069092658552 2716.9049727409906
2897.2112558576714 2739.229335358994
2653.1555867277066 1609.8629624585067
1318.7436823713706 2715.744648451993
2761.420155750764 2617.343493836098
3745.7581384976384 1584.3614066470814

966.6459818945095 2745.7352944929635
3208.0404339816214 1591.9206341514362
1942.8933224607213 2698.481727958439
803.4917280032537 2808.764470658812
3222.738483220348 1591.817898527842
979.7979967741423 2746.5216945298225
3339.3696889302037 2610.533259682583
3235.07978594224 1591.0907309193003
1958.9212833407612 2699.6319905737573
3350.5772589828885 2612.0609391276394
814.8418104422176 2811.2207321267556
992.4936016458798 2747.484803492282
826.7808540481037 2811.43441067444
1006.6292045124437 2747.58091127426
1976.2700908170386 2699.2055307171513
3361.80058655255 2614.7476361437875
3249.023238315659 1591.0132004771244
1991.377544008415 2698.748609555048
3373.2895515960768 2616.405751413453
3262.200167634844 1590.9680385421723
1018.8884336756355 2748.442192452687
838.4257055490523 2812.388351568352
2007.3482736103003 2698.547479758569
3275.4943177870105 1590.77585450572
3384.841951798162 2618.888092707874
851.8396176431529 2813.84427014926
1034.7696876171162 2748.836993162119
3397.316012

2684.9486639412953 2675.209193685908
2700.881488844088 2674.9586499161815
3695.4998747287445 1573.1130197651337
1631.6840937319523 2750.4229135611827
439.5174330749255 2459.8102984004568
1383.960693673379 2839.5596841792103
3702.88387145402 1572.3024464356063
1650.862590981788 2750.1810133864815
448.667035862901 2461.2366690362805
2717.4533739173007 2675.760213129997
1398.920314067983 2841.1629169724283
455.9180261734821 2460.9107756470366
2733.549021847629 2676.636533898897
1414.3562191709711 2841.4289922456896
1666.9134097906094 2752.185422636208
3709.9038944033846 1571.186373618259
2751.040980300381 2676.4032006511866
1430.124832321129 2841.761245302311
3718.4346301306987 1570.916027763671
464.2264417865829 2460.2144645242292
1686.6984093697945 2750.9722415403476
2767.8886243979177 2675.735472407284
3724.257783969336 1570.9387910748333
477.35848492467267 2462.071033407588
1702.804788244015 2753.0543254634094
1447.0755000131032 2842.0181722578845
487.1143592735518 2462.538548910718
2

1940.1700342145866 1442.7636511193973
3449.665454557902 2652.7830211494384
840.7143696902019 2469.5030332754004
2246.622547594818 2746.684151089191
1968.5808763948812 2859.8510650266317
2264.2734754525427 2747.306516265081
1957.7872280152287 1443.7382029315618
1988.4241822420704 2859.949391806042
855.8067649157833 2470.022701840929
3461.36900079929 2650.664372622066
2005.8555804413324 2859.793819979648
870.0536832677353 2471.1632977725803
2282.046511029594 2747.533528184959
3471.186868568094 2651.0694803506212
1975.6772006427495 1444.755398367121
2024.7698340221903 2861.1380309172328
883.3107755714125 2471.8320964497243
3482.348503147995 2649.7406633991513
2298.2767374835203 2747.357559863961
1993.9854257219147 1445.2394102380795
2036.821311782203 2861.3209365157363
2319.475918949961 2746.1330928577977
2012.7091331498973 1446.3172074375207
896.7924434015697 2471.140990217671
3490.957382592356 2650.3518680594902
912.7041465541013 2471.8241244664155
3501.739295641525 2649.389082362853
20

1692.2828010834446 1439.337138571088
2742.2880948945826 2893.35661915476
3051.4213975218777 2760.4957451300415
1637.9222920203488 2478.6142967706014
1656.3777271855652 2479.461390164604
2807.5584889387883 1449.3695593514374
3066.7149035142756 2762.2687414396705
1710.3198675761805 1438.9739882086865
1671.6492417326804 2478.741015694635
3078.1295522561186 2762.713753567573
1728.2862531680894 1438.6337440589473
2823.5888539545676 1449.037694615346
1747.913015568638 1438.8814016075307
2839.9164141548813 1449.3627699210601
3090.5889307878224 2763.139233031396
1690.226426344028 2478.8895944218225
1708.0217648654861 2478.5545103052254
2856.9593504286513 1453.4513035408304
1766.9510024553065 1438.6930191750153
3102.669618846875 2763.6581914638787
1726.9091967869872 2480.092593325937
1785.3042564433779 1438.4066320329946
2874.326089822696 1455.576519530608
3115.0271707482707 2763.514244729098
1803.6992358854745 1438.131635790985
3126.3242528973797 2763.9845760055523
1745.5589004821518 2481.9777